### Import Libraries

In [ ]:
%matplotlib inline
import numpy as np
import sys
import nibabel as nib
from nilearn import plotting

### Download Data

In [ ]:
! ./download-data.sh

### Run Searchlight SRM

In [ ]:
! mpirun -n 4 python3 searchlight_srm_example.py

### Get Whole Brain Mask

In [ ]:
wbmask_fname = 'data/MNI152_T1_3mm_brain_mask.nii'
mask = nib.load(wbmask_fname)
affine = mask.affine
mask = mask.get_data()
zeromaskidx = mask <= 0

### Get Accuracy Map

In [ ]:
acc_path = 'data/searchlight_srm_tsm_acc.npz'
acc_file = np.load(acc_path)
acc_map = acc_file['acc']

### Map to MNI Space

In [ ]:
# Parameters
ds = 6 #the dataset we provided was downsampled by 6
mni_shape = [61,73,61]
min_size  = [1,7,0] # min and max of nonzeros
max_size  = [59,65,58]
acc_mni = np.zeros_like(mask)
# mapping (make sure length of each dimension is a multiple of 'ds')
acc_mni[min_size[0]-1:max_size[0]+1, min_size[1]-1:max_size[1]+1, min_size[2]:max_size[2]+2] = \
acc_map.repeat(ds, axis=0).repeat(ds, axis=1).repeat(ds, axis=2)

# if no downsampling
#acc_mni[min_size[0]:max_size[0], min_size[1]:max_size[1], min_size[2]:max_size[2]] = acc_map

# mask
acc_mni[zeromaskidx] = 0

### Save accuracy map

In [ ]:
out_path = 'data/searchlight_srm_tsm_acc_mni.nii'
acc_img = nib.Nifti1Image(acc_mni, affine)
nib.save(acc_img, out_path)

### Visualize Results

In [ ]:
plotting.plot_stat_map(out_path)